In [ ]:
# Databricks notebook source
# MAGIC %pip install --upgrade numba
# MAGIC %pip install --upgrade pybind11
# MAGIC #%pip install nmslib
# MAGIC #%pip install 'nmslib @ git+https://github.com/nmslib/nmslib.git#egg=nmslib&subdirectory=python_bindings'
# MAGIC %pip install -U genieclust
# MAGIC %pip install -U FlagEmbedding
# MAGIC %pip install -U pacmap

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../libs'))
sys.path.append(os.path.abspath('../'))
from init import spark
from IPython.display import display

24/09/04 08:32:45 WARN Utils: Your hostname, Ankbot resolves to a loopback address: 127.0.1.1; using 192.168.1.140 instead (on interface wlo1)
24/09/04 08:32:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/taoufik/.ivy2/cache
The jars for the packages stored in: /home/taoufik/.ivy2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6db1dbb8-abe2-42ab-8c86-7f8c435577bc;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
:: resolution report :: resolve 1164ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.13;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   1   |   1   |   0   ||   3   |   0   |
	---------------------------------------------------------------------

:: problems summary ::
:::: ERRORS
	unknown resolver null


:: USE VERBOSE OR DEBUG MESSAGE LEVEL FOR MORE DETAILS
:: retrieving :: org.apache.spark#spark-submit-parent-6db1dbb8-abe2-42ab-8c86-7f8c435577bc
	confs: [default]
	0 artifacts copied, 3 already ret

In [2]:
from context import initialize_context

context = initialize_context()

(context)

{'tenant_id': 'root',
 'workspace_id': 'root',
 'table_prefix': '',
 'project_id': 'project1',
 'import_id': 'import1',
 'job_id': 'job1'}

In [3]:
from dataproject import DataProject
data_project = DataProject(context["project_id"])
configuration = data_project.getConfiguration()

content_clusters = [6, 18, 54, 162, 486]

(content_clusters)

[6, 18, 54, 162, 486]

In [4]:
contents = spark.sql(f"""SELECT * FROM norm_contents WHERE project_id='{context["project_id"]}' AND content_group='{context["import_id"]}'""")
contents = contents.limit(1000)
contents.show()
display(contents.count())

+----------+-------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|project_id|content_group|                  id|         original_id|line|             content|               dim_1|               dim_2|               dim_3|dim_4|dim_5|dim_6|dim_7|dim_8|dim_9|dim_10|dim_11|dim_12|dim_13|dim_14|dim_15|dim_16|dim_17|dim_18|dim_19|dim_20|dim_21|dim_22|dim_23|dim_24|dim_25|
+----------+-------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|  project1|      import1|00b4b3d5-62ff-48c...|tag:search.twitte...|   0|« si vous

1000

In [5]:
# MAGIC %pip install peft
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3') #, use_fp16=True)
model


2024-09-04 08:33:12.457758: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 08:33:12.474042: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 08:33:12.592649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 08:33:13.130974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/taoufik/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
import pandas as pd
import pacmap
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import ArrayType, FloatType

"""def compute_embeddings():
    content_texts = [str(content['content']) for content in contents.select(
        col('content')
    ).collect()]

    return  pd.DataFrame(
        model.encode(content_texts, return_dense=True)["dense_vecs"]
    )


content_embeddings =  compute_embeddings()
"""

# Assuming `model.encode` is a function that works on a list of strings and returns a list of dense vectors

# Define the UDF using pandas_udf
# @pandas_udf("array<float>", PandasUDFType.SCALAR_ITER)
#@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR)
@pandas_udf(ArrayType(FloatType()))
def encode_udf(content_series: pd.Series) -> pd.Series:
    # Convert the series to a list of strings
    content_list = content_series.astype(str).tolist()
    
    # Encode the list of strings to get a list of dense vectors
    dense_vecs = model.encode(content_list, return_dense=True)["dense_vecs"]
    
    # Ensure each dense_vec is flattened and converted to a list
    flat_dense_vecs = [vec.tolist() for vec in dense_vecs]
    
    # Return the results as a pandas Series
    return pd.Series(flat_dense_vecs)
    
# Assuming you have a Spark session and contents DataFrame
# Apply the UDF to the 'content' column to get the embeddings
content_embeddings_df = contents.withColumn("embedding", encode_udf(col("content")))

# Collect the embeddings into a local DataFrame
content_embeddings = content_embeddings_df.select("embedding").collect()

2024-09-04 08:33:27.768497: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 08:33:27.769611: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 08:33:27.790384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 08:33:28.100079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/taoufik/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.w

In [9]:
clustered_content

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   1,   0, ...,   1,   1,   1],
       ...,
       [  0,   1,   2, ..., 483, 466, 466],
       [  0,   1,   2, ..., 484, 467, 467],
       [  0,   1,   2, ..., 485, 468, 468]])

In [8]:
import functools
import genieclust
import numpy as np
from pyspark.sql.functions import col, lit, to_number

def compute_content_clustering(input, clusters):
    return genieclust.Genie(
            n_clusters=clusters, cast_float32=True, gini_threshold=0.2,
            affinity="cosinesimil", exact=False, compute_all_cuts=True, compute_full_tree=True
        ).fit_predict(
           input
        )

def compute_content_clustering_hierarchy(clustered_content):
    return pd.DataFrame(
        np.transpose( clustered_content[content_clusters, :]),
        columns=[f'k{content_cluster}' for content_cluster in content_clusters]
    ).drop_duplicates()

content_embeddings_array = np.array([row.embedding for row in content_embeddings], dtype=np.float32)

clustered_content = compute_content_clustering(content_embeddings_array, content_clusters[-1])
cluster_hierarchy = compute_content_clustering_hierarchy(clustered_content)


display(cluster_hierarchy)

/home/taoufik/.local/lib/python3.11/site-packages/genieclust/genie.py:193: UserWarning: `compute_full_tree` is only available when `M` = 1 and `exact` is True
  warnings.warn("`compute_full_tree` is only available when `M` = 1 "


,k6,k18,k54,k162,k486
0,0,0,0,0,0
1,1,1,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3
4,2,2,2,2,4
...,...,...,...,...,...
986,5,17,53,146,481
987,5,16,50,159,482
989,5,17,53,146,483
991,5,16,50,160,484


In [21]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

def compute_content_topics(contents, content_clusters, index, clustered_content):
    content_ids = contents.select(col("id")).toPandas()
    content_ids["content_topic"] = clustered_content[content_clusters[index], :]
    if index == 0:
        content_ids["content_topic_ancestor"] = np.repeat(None, len(content_ids.index))
    else:
        content_ids["content_topic_ancestor"] = clustered_content[content_clusters[index - 1], :]
    return content_ids

content_topics = [[content_cluster, compute_content_topics(contents, content_clusters, index, clustered_content)] for [index, content_cluster] in enumerate(content_clusters)]

spark.sql(f"""
    DELETE FROM comp_content_clusterings
    WHERE project_id='{context["project_id"]}' AND
          content_group='{context["import_id"]}'
""")

schema = StructType([
    StructField("id", StringType(), False),
    StructField("content_topic", IntegerType(), True),
    StructField("content_topic_ancestor", IntegerType(), True)
])

for [content_cluster, content_topic] in content_topics:
    final = spark.createDataFrame(content_topic, schema=schema).select(
        lit(context["project_id"]).alias("project_id"),
        lit(context["import_id"]).alias("content_group"),
        col("id").alias("content_id"),
        col("content_topic").cast("INTEGER"),
        col("content_topic_ancestor").cast("INTEGER"),
        lit(content_cluster).alias("content_topic_clusters").cast("INTEGER")
    )
    
    final.writeTo(f"""comp_content_clusterings""").using("delta").append()

    display(final)

DataFrame[project_id: string, content_group: string, content_id: string, content_topic: int, content_topic_ancestor: int, content_topic_clusters: int]

DataFrame[project_id: string, content_group: string, content_id: string, content_topic: int, content_topic_ancestor: int, content_topic_clusters: int]

DataFrame[project_id: string, content_group: string, content_id: string, content_topic: int, content_topic_ancestor: int, content_topic_clusters: int]

DataFrame[project_id: string, content_group: string, content_id: string, content_topic: int, content_topic_ancestor: int, content_topic_clusters: int]

DataFrame[project_id: string, content_group: string, content_id: string, content_topic: int, content_topic_ancestor: int, content_topic_clusters: int]

In [ ]:
#workspace.progress(context['job_id'])

In [37]:
#df = spark.sql("select * from norm_content_dims where ...")
df = spark.sql("""SELECT
  norm_content_dims.data_1,
  comp_content_clusterings.content_topic,
  COUNT(norm_contents.id) AS count
FROM norm_content_dims
JOIN norm_contents ON norm_content_dims.id = norm_contents.dim_2
JOIN comp_content_clusterings ON norm_contents.id = comp_content_clusterings.content_id
WHERE norm_contents.project_id = 'project1' AND
      norm_content_dims.project_id = 'project1' AND
      comp_content_clusterings.project_id = 'project1' AND
      dimension = 2 AND
      comp_content_clusterings.content_topic_clusters = 18
GROUP BY content_topic, data_1""")
df.show()
df.select("content_topic").distinct().show()
spark.sql("select content_topic_clusters from comp_content_clusterings").distinct().show()

+----------+-------------+-----+
|    data_1|content_topic|count|
+----------+-------------+-----+
|2023-07-12|            1|    3|
|2023-10-31|            0|    1|
|2023-03-29|            9|    1|
|2023-05-28|            5|    1|
|2023-08-21|            2|    1|
|2023-03-02|            6|    3|
|2023-08-15|            3|    1|
|2023-01-31|            4|   24|
|2023-07-12|            5|   21|
|2023-01-21|            0|    4|
|2023-03-24|            4|    1|
|2023-09-23|            6|    1|
|2023-08-26|            3|    1|
|2023-03-31|            6|    1|
|2023-02-18|            4|    4|
|2023-01-31|            5|    4|
|2023-09-06|            5|    1|
|2023-09-05|            8|    1|
|2022-10-02|            7|    1|
|2023-06-13|            0|    1|
+----------+-------------+-----+
only showing top 20 rows

+-------------+
|content_topic|
+-------------+
|            1|
|            6|
|            3|
|            5|
|            9|
|            4|
|            8|
|            7|
|     

+----------------------+
|content_topic_clusters|
+----------------------+
|                    54|
|                   486|
|                    18|
|                   162|
|                     6|
+----------------------+



24/09/04 15:21:13 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-c6285f8a-7870-43c7-8dc9-89c9bf2908d5. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-c6285f8a-7870-43c7-8dc9-89c9bf2908d5
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.ArrayOps$.foreach$